In [4]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Get the absolute path of the current notebook
notebook_path = Path().resolve()

# Get the project root directory (which is the parent of the 'notebooks' directory)
project_root = notebook_path.parent

# Add BOTH the project root and the src directory to the Python path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
if str(project_root / 'src') not in sys.path:
    sys.path.append(str(project_root / 'src'))

# Now, we can import our modules
from src.data_handling import DataHandler

In [3]:
preds_dir = "../2020-results-20251023/preds/"
preds = {
    "ridge_base": pd.read_csv(preds_dir + "ridge_base_preds.csv"),
    "ridge_pca": pd.read_csv(preds_dir + "ridge_pca_preds.csv"),
    "xgboost": pd.read_csv(preds_dir + "xgb_base_preds.csv"),
    "softmax": pd.read_csv(preds_dir + "softmax_preds.csv"),
    "mlp_1": pd.read_csv(preds_dir + "mlp_depth1_preds.csv"),
    "mlp_2": pd.read_csv(preds_dir + "mlp_depth2_preds.csv"),
    "mlp_3": pd.read_csv(preds_dir + "mlp_depth3_preds.csv")
}

In [5]:
dh = DataHandler()

DataHandler initialized - Using 52 features - Test year: 2020


In [6]:
dir(dh)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_create_dataloader',
 '_create_dmatrix',
 '_create_tensors',
 '_fit_scaler',
 '_fit_wpca',
 '_load_data',
 '_make_dirs',
 'cv_pca',
 'cv_scalers',
 'cv_year_pairs',
 'data_dir',
 'features',
 'final_pca',
 'final_scaler',
 'get_mlp_data',
 'get_ridge_data',
 'get_xgb_data',
 'idx',
 'models_dir',
 'n_features',
 'optuna_dir',
 'preds_dir',
 'raw_data',
 'results_dir',
 'targets',
 'test_year',
 'train_years',
 'years']

In [12]:
_, y, wts = dh.get_ridge_data(task="test")

In [53]:
def weighted_metrics(p_true, p_pred, weights):
    """
    Compute comprehensive weighted metrics for voting predictions.
    
    Returns:
        dict with keys:
        - p_dem, p_rep, p_other, p_nonvoter: weighted avg probabilities
        - log_odds_dem_vs_rep: log(P(dem)/P(rep))
        - weighted_entropy: weighted average entropy of predicted distribution
        - weighted_kl: weighted average KL divergence
        - weighted_kl_percent: weighted average KL divergence as % of entropy
        - weighted_accuracy: weighted fraction of correct county winner predictions
    """
    p_true = np.clip(p_true, 1e-10, 1)
    p_pred = np.clip(p_pred, 1e-10, 1)
    
    # Flatten weights to ensure 1D
    weights = weights.flatten()
    
    # Compute weighted average probabilities for each class
    # Assumes columns are [P(dem), P(rep), P(other), P(nonvoter)]
    weighted_probs = np.sum(weights[:, np.newaxis] * p_pred, axis=0) / np.sum(weights)
    
    # Compute log-odds: log(P(dem)/P(rep))
    log_odds = np.log(weighted_probs[0] / weighted_probs[1])
    
    # Compute entropy of predicted distribution for each sample
    sample_pred_entropies = -np.sum(p_pred * np.log(p_pred), axis=1)
    weighted_entropy = np.sum(weights * sample_pred_entropies) / np.sum(weights)
    
    # Compute KL divergence and KL-div-percent for each sample
    sample_kls = np.sum(p_true * np.log(p_true / p_pred), axis=1)
    sample_true_entropies = -np.sum(p_true * np.log(p_true), axis=1)
    sample_kl_percents = sample_kls / sample_true_entropies
    
    # Weight and average
    weighted_kl = np.sum(weights * sample_kls) / np.sum(weights)
    weighted_kl_percent = np.sum(weights * sample_kl_percents) / np.sum(weights)
    
    # Compute weighted accuracy: fraction of correct county winner predictions
    true_log_odds = np.log(p_true[:, 0] / p_true[:, 1])
    pred_log_odds = np.log(p_pred[:, 0] / p_pred[:, 1])
    true_winners = (true_log_odds > 0).astype(int)  # 1 for Dem, 0 for Rep
    pred_winners = (pred_log_odds > 0).astype(int)
    correct = (true_winners == pred_winners).astype(int)
    weighted_accuracy = np.sum(weights * correct) / np.sum(weights)
    
    return {
        'p_dem': weighted_probs[0],
        'p_rep': weighted_probs[1],
        'p_other': weighted_probs[2],
        'p_nonvoter': weighted_probs[3],
        'log_odds_dem_vs_rep': log_odds,
        'weighted_entropy': weighted_entropy,
        'weighted_kl': weighted_kl,
        'weighted_kl_percent': weighted_kl_percent,
        'weighted_accuracy': weighted_accuracy
    }

def compute_true_distribution(p_true, weights):
    """Compute the weighted average of the true distribution."""
    p_true = np.clip(p_true, 1e-10, 1)
    weights = weights.flatten()
    
    weighted_probs = np.sum(weights[:, np.newaxis] * p_true, axis=0) / np.sum(weights)
    log_odds = np.log(weighted_probs[0] / weighted_probs[1])
    
    # Compute weighted entropy of the true distribution
    sample_entropies = -np.sum(p_true * np.log(p_true), axis=1)
    weighted_entropy = np.sum(weights * sample_entropies) / np.sum(weights)
    
    return {
        'model': 'true values',
        'p_dem': weighted_probs[0],
        'p_rep': weighted_probs[1],
        'p_other': weighted_probs[2],
        'p_nonvoter': weighted_probs[3],
        'log_odds_dem_vs_rep': log_odds,
        'weighted_entropy': weighted_entropy,
        'weighted_kl': 0.0,
        'weighted_kl_percent': 0.0,
        'weighted_accuracy': 1.0  # Perfect accuracy for true values
    }

In [54]:
# Create results dataframe with true values at top
rows = []

# Add true values row first
rows.append(compute_true_distribution(y, wts))

# Add model predictions
for model_name, pred_df in preds.items():
    metrics = weighted_metrics(y, pred_df.values, wts)
    row = {'model': model_name}
    row.update(metrics)
    rows.append(row)

results_df = pd.DataFrame(rows)

In [55]:
results_df

,model,p_dem,p_rep,p_other,p_nonvoter,log_odds_dem_vs_rep,weighted_entropy,weighted_kl,weighted_kl_percent,weighted_accuracy
0,true values,0.243365,0.225337,0.009005,0.522294,0.076964,1.020586,0.000000,0.000000,1.000000
1,ridge_base,0.207072,0.180182,0.029945,0.582816,0.139097,1.023213,0.038300,0.040943,0.895151
2,ridge_pca,0.207072,0.180173,0.029898,0.582872,0.139149,1.023103,0.038268,0.040913,0.895151
3,xgboost,0.208658,0.181292,0.029223,0.580827,0.140588,1.018436,0.030040,0.030323,0.925208
4,softmax,0.226287,0.173461,0.042423,0.557829,0.265851,1.061015,0.039892,0.041112,0.902946
5,mlp_1,0.210196,0.170095,0.029840,0.589869,0.211687,1.003226,0.035628,0.036142,0.912668
6,mlp_2,0.219747,0.165085,0.039912,0.575256,0.286018,1.047299,0.041737,0.041889,0.891164
7,mlp_3,0.209622,0.195179,0.034323,0.560876,0.071391,1.065834,0.029262,0.029748,0.916811
